In [3]:
from omegaconf import OmegaConf as om
from omegaconf import DictConfig
from typing import cast
import sys 
import os
sys.path.append(os.path.abspath(".."))

#from main import build_model
import src.create_bert as bert_module
import src.create_model as model_module
from transformers import AutoTokenizer, Trainer, TrainingArguments



yaml_path = "../yamls/pretrain/dna_monarch-mixer-pretrain-786dim-80m-parameters.yaml"

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")


with open(yaml_path) as f:
    cfg = om.load(f)
cfg = cast(DictConfig, cfg)
print(cfg.max_duration)
model = model_module.create_model(cfg.model.get("model_config"))
#model = bert_module.create_bert_mlm(model_config = cfg.model.get("model_config", None))

100000ba
Using Monarch Mixer for Sequence Mixing: True
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [4]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6)

In [5]:
from datasets import load_from_disk
from collate import DataCollatorForLanguageModelingSpan
from torch.utils.data import DataLoader

#Load model directly
%env WANDB_PROJECT=speciesembedding



dataset = load_from_disk("../fullds_speciesembed")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])
data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

dataloader = DataLoader(dataset, batch_size=1, collate_fn=data_collator)

sample = next(iter(dataloader))

env: WANDB_PROJECT=speciesembedding


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


IndexError: tuple index out of range

In [ ]:
sample = next(iter(dataloader))
training_args = TrainingArguments(
    output_dir="./results/correct_model_1sample",
    
    max_steps=100000,
    
    seed=17,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    
    logging_strategy="steps",
    logging_steps=1,
    
    remove_unused_columns=False,
    evaluation_strategy="no",
    
    #dataloader_num_workers=0,
    run_name="repo_model_1sample",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)


trainer.train()
tokenizer.vocab_size

print(sample['species_id'].shape)
print(sample['input_ids'].shape)

/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


env: WANDB_PROJECT=speciesembedding


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luluhu (luluh). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,8.865500
2,8.812700
3,8.761200
4,8.789400
5,8.489200
6,8.797500
7,8.857600
8,8.830600
9,8.805000
10,8.694400


Checkpoint destination directory ./results/correct_model_1sample/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Removed shared tensor {'bert.encoder.layer.4.attention.filter_fn2.implicit_filter_rev.3.freq', 'bert.encoder.layer.0.attention.filter_fn.implicit_filter_rev.1.freq', 'bert.encoder.layer.11.attention.filter_fn.implicit_filter_rev.1.freq', 'bert.encoder.layer.3.attention.filter_fn.implicit_filter_rev.1.freq', 'bert.encoder.layer.2.attention.filter_fn2.implicit_filter_rev.5.freq', 'bert.encoder.layer.5.attention.filter_fn.implicit_filter_rev.5.freq', 'bert.encoder.layer.8.attention.filter_fn2.implicit_filter_rev.3.freq', 'bert.encoder.layer.4.attention.filter_fn2.implicit_filter.5.freq', 'bert.encoder.layer.11.attention.filter_fn2.implicit_filter_rev.5.freq', 'bert.encoder.layer.4.attention.filter_fn.implicit_filter_rev.1.freq', 'bert.encoder.layer.10.attention.filter_fn.implicit_filter.3.freq', 'bert.encoder.layer.0.attenti

In [10]:
model(**sample)

MaskedLMOutput(loss=tensor(8.7274, grad_fn=<NllLossBackward0>), logits=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset["train"], batch_size=1, shuffle=True, collate_fn=data_collator)
for step, inputs in enumerate(train_dataloader):
    inputs = {
        "args": inputs
    }
    model(**inputs)

TypeError: BertForMaskedLM.forward() got an unexpected keyword argument 'args'